In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.layers import MaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


2024-08-28 16:37:28.277851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 16:37:28.293872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 16:37:28.298658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 16:37:28.311839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 16:37:29.375356: W tensorflow/compiler/tf2

In [2]:
# Load data
df = pd.read_csv('/workspaces/Futures-First/Prediction/TCN/data/TVSMOTOR.csv', parse_dates=['Date'], index_col='Date')

# Sort data by date
df.sort_index(inplace=True)

# Normalize the data
scaler = MinMaxScaler()
df['Price'] = scaler.fit_transform(df[['Price']])

# Prepare the dataset
def create_dataset(series, look_back=1):
    X, Y = [], []
    for i in range(len(series)-look_back):
        X.append(series[i:(i+look_back)])
        Y.append(series[i + look_back])
    return np.array(X), np.array(Y)

look_back = 10  # Number of past days to use for prediction
X, Y = create_dataset(df['Price'].values, look_back)

# Reshape data to fit the TCN model
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [3]:
model = Sequential()

# Add TCN layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='causal', input_shape=(look_back, 1)))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='causal'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/100


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0120 - val_loss: 0.0015
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3990e-04 - val_loss: 0.0010
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8525e-04 - val_loss: 0.0011
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4531e-04 - val_loss: 7.7543e-04
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1091e-04 - val_loss: 8.0656e-04
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7222e-04 - val_loss: 0.0019
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3278e-04 - val_loss: 0.0019
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4487e-04 - val_loss: 0.0016
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9176e-04 - val_loss: 0.0010
Epoch 10/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1803e-04 - val_loss: 0.0012
Epoch 11/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2271e-04 - val_loss: 0.0014
Epoch 12/100
62/62 ━━━━━━━━

In [4]:
# Predict on the test set
predictions = model.predict(X_test)

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate prediction for the next day
last_days = df['Price'].values[-look_back:]
last_days = last_days.reshape(1, look_back, 1)
next_day_prediction = model.predict(last_days)
next_day_price = scaler.inverse_transform(next_day_prediction)

print("Predicted price for the next day:", next_day_price)


 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted price for the next day: [[2405.5667]]
